In [1]:
import pandas as pd
import pickle
import re
import unidecode

In [2]:
df_matches = pd.read_pickle("../Data/df_matches.pickle")

In [3]:
df_matches.head(10)

,COMPANY,ADDED_MODIFIED,DOLLAR_AMOUNT_FLOAT,MONEYPRIZE_RECEIVED,DONATION_DETAILS,YEAR,DON_DETAIL_AMOUNT,DOLLAR_EQUIVALENT_AMOUNT,DON_DETAIL_TXT,ORGANIZATION,...,OU,POSTE_FND,PROJETS_PRIVILEGIES,TEL,TOTAL_ANNUEL,WEB,MUNICIPALITY_FND,PROVINCE_FND,POSTALCODE_FND,STREET_FND
0,VIA,2010-01-19,0.0,NaN,trip for 4 Montreal/Windsor $2600 value,2010,2600.0,2600.0,trip for 4 montreal/windsor value,VIA RAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VIA,2011-01-01,0.0,Yes,trip for 4 Montreal/Windsor $2600 value,2011,2600.0,2600.0,trip for 4 montreal/windsor value,VIA RAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VIA,2012-01-09,0.0,Yes,trip for 4 Montreal/Windsor $2251 value,2012,2251.0,2251.0,trip for 4 montreal/windsor value,VIA RAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VIA,2012-06-12,0.0,Yes,trip for 4 Montreal/Windsor $2251 value,2013,2251.0,2251.0,trip for 4 montreal/windsor value,VIA RAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,VIA,2014-03-04,0.0,Yes,Silver,2014,NaN,0.0,silver,VIA RAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,VIA,2014-12-31,0.0,Yes,Raffle prize - full page ad,2015,NaN,0.0,raffle prize - full page ad,VIA RAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ULTRAMAR,2010-01-19,500.0,yes,Silver - $500 in gas coupons,2010,500.0,500.0,silver - in gas coupons,ULTRAMAR,...,NaN,NaN,"Activité bénéfice annuelle (tournois, événemen...",NaN,1 173 800 :$,http://www.ultramarcst.ca -ou- https://www.ul...,montréal,QC,H3B0C9,"1155, boul. rené-lévesque o."
7,ULTRAMAR,2011-01-10,0.0,Yes,Silver - $500 in gas coupons,2011,500.0,500.0,silver - in gas coupons,ULTRAMAR,...,NaN,NaN,"Activité bénéfice annuelle (tournois, événemen...",NaN,1 173 800 :$,http://www.ultramarcst.ca -ou- https://www.ul...,montréal,QC,H3B0C9,"1155, boul. rené-lévesque o."
8,ULTRAMAR,2012-01-10,0.0,Yes,Silver - $500 in gas coupons,2012,500.0,500.0,silver - in gas coupons,ULTRAMAR,...,NaN,NaN,"Activité bénéfice annuelle (tournois, événemen...",NaN,1 173 800 :$,http://www.ultramarcst.ca -ou- https://www.ul...,montréal,QC,H3B0C9,"1155, boul. rené-lévesque o."
9,ULTRAMAR,2012-05-12,0.0,Yes,Silver - $500 in gas coupons,2013,500.0,500.0,silver - in gas coupons,ULTRAMAR,...,NaN,NaN,"Activité bénéfice annuelle (tournois, événemen...",NaN,1 173 800 :$,http://www.ultramarcst.ca -ou- https://www.ul...,montréal,QC,H3B0C9,"1155, boul. rené-lévesque o."


In [4]:
def clean_col_names(df_):

    df = df_.copy()
    cols = df.columns.to_list()
    new_cols=[]
    for col in cols:
        
        col = re.sub(' +', ' ', col)
        col = col.replace(' ','_')
        col = col.replace('/','')
        col = col.replace('$','dollar')
        col = col.upper()
        new_cols.append(col)

    df.columns = new_cols

    return df
    
def stdr_names(series_original):
    '''Clean company names. To be used in the different files so names are likelier to be matched'''
    series = series_original.copy()
    series = series.astype(str)
    
    series = series.str.upper()
    series = series.str.strip()

    series = series.replace(r'\s+', ' ', regex=True)
    series = series.str.replace(r'[^\w\s]+', '', regex=True)


    series = series.apply(lambda x: unidecode.unidecode(x))

    series = series.str.replace(' QUEBEC ', '')
    series = series.str.replace('CANADA', '')
    series = series.str.replace(' MONTREAL ', '')
    series = series.str.replace('MONTREAL ', '')
    series = series.str.replace(' MONTREAL', '')
    series = series.str.replace(' INC', '')
    series = series.str.replace(' INC ', '')
    series = series.str.replace(' CIE ', '')
    series = series.str.replace(' CIE', '')
    series = series.str.replace(' LTEE', '')
    series = series.str.replace('CORPORATION', '')
    series = series.str.replace('INTERNATIONAL', '')
    return series

In [5]:
df_donors = pd.read_csv('../Data/past_donors_clean.csv', parse_dates=[1])
df_orgs = pd.read_excel('../Data/Organizations_stg.xlsx')

df_donors=clean_col_names(df_donors)
df_orgs=clean_col_names(df_orgs)

# now we read the matches after reviews them and this will become our key to match both sources
df_matches = pd.read_excel('../Data/matched_orgs_k4kreview.xlsx')

df_donors.COMPANY = stdr_names(df_donors.COMPANY)
df_orgs.NAME = stdr_names(df_orgs.NAME)

In [6]:
df_orgs = df_orgs.drop(columns = ['ISFOUNDATION'])

df_fonds = pd.read_excel('../Data/Foundations_stg.xlsx')
df_fonds = clean_col_names(df_fonds)
df_fonds['ISFOUNDATION'] = 1

df_orgs = df_orgs.merge(df_fonds[['ID', 'ISFOUNDATION']], how='left', on='ID')
df_orgs.ISFOUNDATION.fillna(0, inplace=True)

## add foundations dataset attributes

In [7]:
df_orgs.columns

Index(['ID', '2E_CONTACT_POUR', 'ADDRESS', 'AVIS', 'CONTACT', 'CONTRIBUTION',
       'COURRIEL', 'DDD', 'DOMAINE_DINTERET', 'FAF', 'FAX', 'FILIALE_DE',
       'LANGUE', 'LIMITES_GEOG', 'N_DE_TEL', 'NAME', 'NBRE_DE_SUCC',
       'NOMBRE_DEMPLOYES', 'NOTE', 'POSTE', 'PRINCIP_FILIALES',
       'SECTEUR_INDUSTRIEL', 'SITE_WEB', 'MUNICIPALITY', 'PROVINCE',
       'POSTALCODE', 'STREET', 'SECTOR_FABRICATION', 'SECTOR_FINANCIERS',
       'SECTOR_COMMERCE', 'SECTOR_CONSTRUCTION', 'SECTOR_TRANSPORT',
       'SECTOR_GESTION', 'SECTOR_SYSTEMS', 'SECTOR_ALIMENTATION',
       'SECTOR_SANTE_ASSURANCE', 'SECTOR_OTHER', 'GENERAL_CHARITY',
       'HEALTH_ARTS_CULTURE', 'COMMUNITY_ENVIRONMENTAL',
       'YOUTH_HEALTH_EDUCATION', 'REGIONAL_SPECIFIC',
       'CANCER_DISEASE_RESEARCH', 'INTEREST_DOMAIN_CLEAN', 'MAIN_TOPIC',
       'MAIN_TOPIC_SCORE', 'SECONDARY_TOPIC', 'SECONDARY_TOPIC_SCR',
       'ISFOUNDATION'],
      dtype='object')

In [8]:
df_matches = df_matches[df_matches.Match == 1] # keeping only matches
df_matches = df_matches.iloc[:, 0:2]
df_matches.columns = ['ORGANIZATION', 'DONOR']

In [9]:
# some past donors have been matched with more than 1 different company in the PDF source, so we need to select and keep only one so the analysis is consistent.
donor_q = df_matches.DONOR.value_counts()
organization_q = df_matches.ORGANIZATION.value_counts()
df1 = df_matches[df_matches.DONOR.isin(donor_q[donor_q > 1].index)].sort_values('DONOR').reset_index(drop=True)

In [10]:
ixs = [1, 2,5,6,10,11,15,16,18, 21,22] 
df1 = df1.iloc[ixs,:].reset_index(drop=True)

In [11]:
# now we concatenate the matches into a single df
df_matches = pd.concat([ df_matches[df_matches.DONOR.isin(donor_q[donor_q <= 1].index)].sort_values('DONOR').reset_index(drop=True),
            df1 ] )

In [12]:
df_donors_match = df_donors.merge(df_matches, how = 'inner', left_on = 'COMPANY', right_on = 'DONOR')


In [13]:
df_donors_match = df_donors_match[['ORGANIZATION']].drop_duplicates()


In [14]:
df_donors_match2 = df_donors_match.merge(df_orgs, how = 'right', left_on = 'ORGANIZATION', right_on = 'NAME')


In [15]:
df = df_donors_match2.drop_duplicates().reset_index(drop=True)


In [16]:
df['FLG_PAST_DONOR'] = df.ORGANIZATION.notna().astype(int)


In [17]:
df = df[ ['NAME',
'FLG_PAST_DONOR',
       'FILIALE_DE', 
       'ISFOUNDATION', 
        'LIMITES_GEOG',
         'NOMBRE_DEMPLOYES',
       'PRINCIP_FILIALES', 
       'SECTOR_FABRICATION', 'SECTOR_FINANCIERS', 'SECTOR_COMMERCE',
       'SECTOR_CONSTRUCTION', 'SECTOR_TRANSPORT', 'SECTOR_GESTION',
       'SECTOR_SYSTEMS', 'SECTOR_ALIMENTATION', 'SECTOR_SANTE_ASSURANCE',
       'SECTOR_OTHER', 'GENERAL_CHARITY',
       'HEALTH_ARTS_CULTURE', 'COMMUNITY_ENVIRONMENTAL',
       'YOUTH_HEALTH_EDUCATION', 'REGIONAL_SPECIFIC',
       'CANCER_DISEASE_RESEARCH'         
         ]]

In [18]:
df['FILIALE_FLG'] = df.FILIALE_DE.isna().astype(int)


In [19]:
df.LIMITES_GEOG = df.LIMITES_GEOG.str.strip('.').astype(str)

mapper = { 'locales, régionales et nationales' :'regional and national',
'locales' : 'locals',
'nationales' : 'nationals',
'régionales' :'regionals',
'nationales, régionales au Québec' : 'regional and national',
'régionales et nationales' : 'regional and national',
'locales, régionales':'local and regional',
'locales et régionales':'local and regional',
'locales, nationales et internationales':'international',
'nan':'nan',
'nationales et internationales':'international'}

df.LIMITES_GEOG = df.LIMITES_GEOG.map(mapper)

In [20]:
df['FILIALE_FLG2'] = df.PRINCIP_FILIALES.isna().astype(int)


In [21]:
df.NOMBRE_DEMPLOYES = df.NOMBRE_DEMPLOYES.astype(str).str.replace(' ','').astype(float)


In [22]:
df = df.drop(columns = ['PRINCIP_FILIALES', 'FILIALE_DE' ])


In [23]:
cols_to_encode = ['LIMITES_GEOG']
dummies = pd.get_dummies(df[cols_to_encode])

df = pd.concat( [df, dummies], axis = 1).drop(columns = cols_to_encode)

In [24]:
from sklearn.preprocessing import StandardScaler

df.NOMBRE_DEMPLOYES.fillna(0, inplace=True)

scaler = StandardScaler()
df.NOMBRE_DEMPLOYES = scaler.fit_transform(df[['NOMBRE_DEMPLOYES']])

In [25]:
df.head().T


,0,1,2,3,4
NAME,3M,A D PREVOST,AAR AICRAFT SERVICES TROISRIVIERES,ABB ALBERT BOB BOB,ABB
FLG_PAST_DONOR,0,0,0,0,0
ISFOUNDATION,0.0,0.0,0.0,0.0,0.0
NOMBRE_DEMPLOYES,-0.224658,-0.193808,-0.150618,-0.206148,0.176397
SECTOR_FABRICATION,1,1,0,1,1
SECTOR_FINANCIERS,0,0,0,0,0
SECTOR_COMMERCE,0,0,0,0,1
SECTOR_CONSTRUCTION,0,1,1,0,0
SECTOR_TRANSPORT,0,0,0,0,1
SECTOR_GESTION,0,0,0,0,0


In [26]:
(df.FLG_PAST_DONOR == 1).sum()

55

# Classification with Synethic Data

## Basic Modeling

In [61]:
def randomSample0(df, proportion):
    """
    Randomly samples 0s according to input proportion
    """
    # proportion = num of 1 / num of 0
    # so num of 0 = num of 1 / proportion
    n_sample = int((df.FLG_PAST_DONOR == 1).sum() / proportion)
    df_merged = pd.concat([df[df.FLG_PAST_DONOR == 1], df[df.FLG_PAST_DONOR == 0].sample(n_sample)])
    return df_merged

In [153]:
df_model = randomSample0(df, 0.08)

In [154]:
df_model.FLG_PAST_DONOR.value_counts()

0    687
1     55
Name: FLG_PAST_DONOR, dtype: int64

In [106]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [107]:
y = df_model.FLG_PAST_DONOR
X = df_model.drop(['FLG_PAST_DONOR', 'NAME'], axis= 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [108]:
base_log_reg = LogisticRegression(random_state = 0, class_weight='balanced').fit(X_train, y_train)
preds = base_log_reg.predict(X_test)

In [109]:
base_report = classification_report(y_test, preds)

In [110]:
print(base_report)
print(accuracy_score(y_test, preds))

              precision    recall  f1-score   support

           0       0.90      0.66      0.76       138
           1       0.02      0.09      0.03        11

    accuracy                           0.62       149
   macro avg       0.46      0.38      0.40       149
weighted avg       0.84      0.62      0.71       149

0.6174496644295302


In [111]:
for i, col_name in enumerate(X_train.columns):
    print(f"{col_name} : {base_log_reg.coef_[0][i]}")

ISFOUNDATION : 1.381450342213512
NOMBRE_DEMPLOYES : 0.12164854274298285
SECTOR_FABRICATION : -1.6131420586441103
SECTOR_FINANCIERS : 0.5526355793849108
SECTOR_COMMERCE : 0.11776478868927087
SECTOR_CONSTRUCTION : 0.575857240839764
SECTOR_TRANSPORT : 0.7800034710575972
SECTOR_GESTION : -0.5758960683822016
SECTOR_SYSTEMS : -0.24417281278985054
SECTOR_ALIMENTATION : 0.31055028090453657
SECTOR_SANTE_ASSURANCE : -0.6561526100366918
SECTOR_OTHER : 0.469833731581597
GENERAL_CHARITY : -0.732948953614824
HEALTH_ARTS_CULTURE : -0.13122398851222955
COMMUNITY_ENVIRONMENTAL : 0.06661395482684643
YOUTH_HEALTH_EDUCATION : 1.0561795446046778
REGIONAL_SPECIFIC : -0.4582545988036256
CANCER_DISEASE_RESEARCH : -0.36124955179349477
FILIALE_FLG : 0.4251067985342568
FILIALE_FLG2 : -0.6063476810744063
LIMITES_GEOG_international : 1.0642916299647243
LIMITES_GEOG_local and regional : -0.41365823835451426
LIMITES_GEOG_locals : -0.49818607770474693
LIMITES_GEOG_nan : 0.6217421611649552
LIMITES_GEOG_nationals : 0.9

In [112]:
df_model.groupby(['FLG_PAST_DONOR', 'ISFOUNDATION']).size().unstack(fill_value=0) / len(df_model)


ISFOUNDATION,0.0,1.0
FLG_PAST_DONOR,,
0,0.877358,0.048518
1,0.057951,0.016173


In [113]:
confusion_matrix(y_test, preds)

array([[91, 47],
       [10,  1]], dtype=int64)

In [114]:
preds

array([0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])

### CV

In [155]:
from sklearn.model_selection import cross_validate
from sklearn import metrics

In [156]:
cv_log_reg_model = LogisticRegression(random_state = 0, class_weight = 'balanced')

In [158]:
cv_log_reg_scores = cross_validate(cv_log_reg_model, X, y, cv=5, scoring=('accuracy','recall','precision','f1'))

In [170]:
print(f"{cv_log_reg_scores['test_accuracy'].mean()} accuracy, standard dev of {cv_log_reg_scores['test_accuracy'].std()}")
print(f"{cv_log_reg_scores['test_recall'].mean()} recall, standard dev of {cv_log_reg_scores['test_recall'].std()}")
print(f"{cv_log_reg_scores['test_precision'].mean()} precision, standard dev of {cv_log_reg_scores['test_precision'].std()}")
print(f"{cv_log_reg_scores['test_f1'].mean()} f1, standard dev of {cv_log_reg_scores['test_f1'].std()}")

0.6780246689642663 accuracy, standard dev of 0.04340193413149914
0.4909090909090909 recall, standard dev of 0.17814470856604933
0.1146929055195233 precision, standard dev of 0.03951196576142959
0.18542374080362198 f1, standard dev of 0.06377800328946133


In [171]:
cv_log_reg_scores

{'fit_time': array([0.01396203, 0.01296496, 0.009974  , 0.01196671, 0.00797868]),
 'score_time': array([0.00398803, 0.004987  , 0.00299215, 0.00498676, 0.00398922]),
 'test_accuracy': array([0.62416107, 0.63758389, 0.68243243, 0.74324324, 0.7027027 ]),
 'test_recall': array([0.27272727, 0.45454545, 0.45454545, 0.45454545, 0.81818182]),
 'test_precision': array([0.05882353, 0.09433962, 0.10869565, 0.13513514, 0.17647059]),
 'test_f1': array([0.09677419, 0.15625   , 0.1754386 , 0.20833333, 0.29032258])}

### RandomForest

In [119]:
from sklearn.ensemble import RandomForestClassifier

In [120]:
rf = RandomForestClassifier(random_state = 0, class_weight = 'balanced').fit(X_train, y_train)
rf_preds = rf.predict(X_test)

In [121]:
rf_report = classification_report(y_test, rf_preds)

H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [122]:
print(rf_report)
print(accuracy_score(y_test, rf_preds))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96       138
           1       0.00      0.00      0.00        11

    accuracy                           0.93       149
   macro avg       0.46      0.50      0.48       149
weighted avg       0.86      0.93      0.89       149

0.9261744966442953


In [172]:
cv_rf_scores = cross_validate(rf, X, y, cv=5, scoring=('accuracy','recall','precision','f1'))

H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetri

In [173]:
print(f"{cv_rf_scores['test_accuracy'].mean()} accuracy, standard dev of {cv_rf_scores['test_accuracy'].std()}")
print(f"{cv_rf_scores['test_recall'].mean()} recall, standard dev of {cv_rf_scores['test_recall'].std()}")
print(f"{cv_rf_scores['test_precision'].mean()} precision, standard dev of {cv_rf_scores['test_precision'].std()}")
print(f"{cv_rf_scores['test_f1'].mean()} f1, standard dev of {cv_rf_scores['test_f1'].std()}")

0.9245329221839288 accuracy, standard dev of 0.002542268656610631
0.0 recall, standard dev of 0.0
0.0 precision, standard dev of 0.0
0.0 f1, standard dev of 0.0


# Synthetic Data

In [40]:
from sdv.tabular import CTGAN
from sdv import SDV

H:\Anaconda\envs\Kurling4Kids\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
df.LIMITES_GEOG_international = df.LIMITES_GEOG_international.astype("int64")
df.LIMITES_GEOG_locals = df.LIMITES_GEOG_locals.astype("int64")
df.LIMITES_GEOG_nan = df.LIMITES_GEOG_nan.astype("int64")
df.LIMITES_GEOG_nationals = df.LIMITES_GEOG_nationals.astype("int64")
df.LIMITES_GEOG_regionals = df.LIMITES_GEOG_regionals.astype("int64")
df['LIMITES_GEOG_local and regional'] = df['LIMITES_GEOG_local and regional'].astype("int64")
df['LIMITES_GEOG_regional and national'] = df['LIMITES_GEOG_regional and national'].astype("int64")

In [42]:
model = CTGAN()
model.fit(df.drop('NAME', axis=1))

H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\mixture\_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\mixture\_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\mixture\_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\mixture\_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\skle

In [140]:
df_synth = model.sample(5000)

In [141]:
df_synth.head()

,FLG_PAST_DONOR,ISFOUNDATION,NOMBRE_DEMPLOYES,SECTOR_FABRICATION,SECTOR_FINANCIERS,SECTOR_COMMERCE,SECTOR_CONSTRUCTION,SECTOR_TRANSPORT,SECTOR_GESTION,SECTOR_SYSTEMS,...,CANCER_DISEASE_RESEARCH,FILIALE_FLG,FILIALE_FLG2,LIMITES_GEOG_international,LIMITES_GEOG_local and regional,LIMITES_GEOG_locals,LIMITES_GEOG_nan,LIMITES_GEOG_nationals,LIMITES_GEOG_regional and national,LIMITES_GEOG_regionals
0,1,0.0,-0.029007,1,0,0,0,0,0,0,...,0.35,0,1,0,1,0,0,1,0,0
1,0,0.0,-0.094031,0,1,0,0,1,0,0,...,0.80,1,1,0,0,0,0,1,0,0
2,0,0.0,-0.136729,1,0,0,0,0,0,0,...,0.01,1,0,0,1,0,0,0,0,0
3,1,1.0,0.360028,1,0,0,0,1,1,0,...,0.05,1,1,0,0,0,0,0,0,1
4,0,0.0,-0.119426,0,0,0,0,0,0,1,...,0.06,0,1,0,0,0,0,0,0,0


In [142]:
df_synth.FLG_PAST_DONOR.value_counts()

0    4702
1     298
Name: FLG_PAST_DONOR, dtype: int64

In [143]:
y_synth = df_synth.FLG_PAST_DONOR
X_synth = df_synth.drop(['FLG_PAST_DONOR'], axis= 1)
X_train_synth, X_test_synth, y_train_synth, y_test_synth = train_test_split(X_synth, y_synth, test_size = 0.2)

In [144]:
synth_log_reg = LogisticRegression(random_state = 0, class_weight = 'balanced').fit(X_train_synth, y_train_synth)
preds_synth = synth_log_reg.predict(X_test_synth)

In [145]:
synth_report = classification_report(y_test_synth, preds_synth)

In [146]:
print(synth_report)
print(accuracy_score(y_test_synth, preds_synth))

              precision    recall  f1-score   support

           0       0.94      0.71      0.81       929
           1       0.10      0.44      0.17        71

    accuracy                           0.69      1000
   macro avg       0.52      0.57      0.49      1000
weighted avg       0.88      0.69      0.76      1000

0.689


In [163]:
cv_log_reg_synth_scores = cross_validate(synth_log_reg, X_synth, y_synth, cv=5, scoring=('accuracy','recall','precision','f1'))

In [169]:
print(f"{cv_log_reg_synth_scores['test_accuracy'].mean()} accuracy, standard dev of {cv_log_reg_synth_scores['test_accuracy'].std()}")
print(f"{cv_log_reg_synth_scores['test_recall'].mean()} recall, standard dev of {cv_log_reg_synth_scores['test_recall'].std()}")
print(f"{cv_log_reg_synth_scores['test_precision'].mean()} precision, standard dev of {cv_log_reg_synth_scores['test_precision'].std()}")
print(f"{cv_log_reg_synth_scores['test_f1'].mean()} f1, standard dev of {cv_log_reg_synth_scores['test_f1'].std()}")

0.6464000000000001 accuracy, standard dev of 0.005003998401278726
0.43254237288135594 recall, standard dev of 0.07864305311378667
0.07435241476914431 precision, standard dev of 0.012152514117228489
0.12688311069359876 f1, standard dev of 0.0210732224845827


### RandomForest

In [147]:
rf_synth = RandomForestClassifier(random_state = 0, class_weight = 'balanced').fit(X_train_synth, y_train_synth)
rf_preds_synth = rf.predict(X_test_synth)

In [148]:
rf_report_synth = classification_report(y_test_synth, rf_preds_synth)

H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [149]:
print(rf_report_synth)
print(accuracy_score(y_test_synth, rf_preds_synth))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96       929
           1       0.00      0.00      0.00        71

    accuracy                           0.93      1000
   macro avg       0.46      0.50      0.48      1000
weighted avg       0.86      0.93      0.89      1000

0.929


In [174]:
cv_rf_synth_scores = cross_validate(rf_synth, X_synth, y_synth, cv=5, scoring=('accuracy','recall','precision','f1'))

H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
H:\Anaconda\envs\Kurling4Kids\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetri

In [175]:
print(f"{cv_rf_synth_scores['test_accuracy'].mean()} accuracy, standard dev of {cv_rf_synth_scores['test_accuracy'].std()}")
print(f"{cv_rf_synth_scores['test_recall'].mean()} recall, standard dev of {cv_rf_synth_scores['test_recall'].std()}")
print(f"{cv_rf_synth_scores['test_precision'].mean()} precision, standard dev of {cv_rf_synth_scores['test_precision'].std()}")
print(f"{cv_rf_synth_scores['test_f1'].mean()} f1, standard dev of {cv_rf_synth_scores['test_f1'].std()}")

0.9404 accuracy, standard dev of 0.0004898979485566361
0.0 recall, standard dev of 0.0
0.0 precision, standard dev of 0.0
0.0 f1, standard dev of 0.0
